In [0]:
my_collection = "Spark The Definitive Guide : Big Data Processing Made Simple"\
  .split(" ")
words = spark.sparkContext.parallelize(my_collection, 2)


In [0]:
 supplementalData = {"Spark":1000, "Definitive":200,
                    "Big":-300, "Simple":100}


suppBroadcast
- broadcast this structure across Spark and reference it 
- value is immutable and is lazily replicated across all nodes in the cluster when we trigger an action

In [0]:
suppBroadcast = spark.sparkContext.broadcast(supplementalData)


In [0]:
suppBroadcast.value


Out[4]: {'Spark': 1000, 'Definitive': 200, 'Big': -300, 'Simple': 100}

transform our RDD using this value
-  create a key–value pair according to the value we might have in the map
- If we lack the value, we will simply replace it with 0

In [0]:

words.map(lambda word: (word, suppBroadcast.value.get(word, 0)))\
  .sortBy(lambda wordPair: wordPair[1])\
  .collect()


Out[5]: [('Big', -300),
 ('The', 0),
 ('Guide', 0),
 (':', 0),
 ('Data', 0),
 ('Processing', 0),
 ('Made', 0),
 ('Simple', 100),
 ('Definitive', 200),
 ('Spark', 1000)]

In [0]:
#use the Dataset API as opposed to the RDD API

path= "/FileStore/tables/part_r_00000_1a9822ba_b8fb_4d8e_844a_ea30d0801b9e_gz-1.parquet"

flights = spark.read\
  .parquet(path)


In [0]:
#  demonstrates creating an unnamed accumulator
accChina = spark.sparkContext.accumulator(0)


**Instantiate the accumulator and register it with a name**

Specify the name of the accumulator in the string value that we pass into the function, or as the second parameter into the register function
- Named accumulators will display in the SparkUI, whereas unnamed ones will not
- Next step is to define the way we add to our accumulator

In [0]:
def accChinaFunc(flight_row):
  destination = flight_row["DEST_COUNTRY_NAME"]
  origin = flight_row["ORIGIN_COUNTRY_NAME"]
  if destination == "China":
    accChina.add(flight_row["count"])
  if origin == "China":
    accChina.add(flight_row["count"])


iterate over every row in our flights dataset via the foreach method
- foreach is an action, and Spark can provide guarantees that perform only inside of actions
- foreach method will run once for each row in the input DataFrame (assuming that we did not filter it) and will run our function against each row, incrementing the accumulator accordingly:

In [0]:
flights.foreach(lambda flight_row: accChinaFunc(flight_row))


In [0]:
# can query it programmatically
accChina.value # 953


Out[11]: 953